In [17]:
import tensorflow as tf
import os
import numpy as np
import IPython.display as display
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import pandas as pd
import random
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from PIL import Image
import cv2
import time 
from skimage.transform import resize




model = tf.keras.models.load_model("animal_cnn_model.h5")
classes = ['bird', 'cat', 'deer', 'dog', 'rabbit', 'raccoon', 'squirrel']

output = []

#cap = cv2.VideoCapture(0) #webcam
cap=cv2.VideoCapture('deer.mp4') #easy video
#cap=cv2.VideoCapture('backyard.mp4') #hard video 
ret1,frame1= cap.read()
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
gray1 = cv2.GaussianBlur(gray1, (19, 19), 1)

#start_time = time.time()

while(cap.isOpened()):
   # elapsed_time = time.time() - start_time
#     if not motion:
#         ret1,frame1= cap.read()
#         gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
#         gray1 = cv2.GaussianBlur(gray1, (19, 19), 1)
        
#     motion = False
    
    
    ret2,frame2=cap.read()
    if not(ret2):
        break
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.GaussianBlur(gray2, (19, 19), 1)
    
    deltaframe=cv2.absdiff(gray1,gray2)
   # cv2.imshow("delta",deltaframe)
    threshold = cv2.threshold(deltaframe, 30, 255, cv2.THRESH_BINARY)[1]
    thresh_frame = cv2.dilate(threshold, None, iterations = 2) 
   # cv2.imshow("treshold",threshold)
    countour,heirarchy = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for i in countour:
        if cv2.contourArea(i) < 3000:
            motion = True
            continue
      
 
        (x, y, w, h) = cv2.boundingRect(i)
        cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 0, 225), 2)
        obj = frame2[y:y+h,x:x+w]
        cv2.imwrite("test.jpg",obj)
        
        
        obj_re = resize(obj, (224,224,3))
        
        
        obj_re = obj_re.reshape((1,) + obj_re.shape)
       # obj = np.array([obj])
        
        
        predicts = model.predict(obj_re)
       # print(predicts)
        index = np.argsort(predicts[0,:])
        
        if predicts[0,index[6]] > .9:
            #print("Most likely class:", classes[index[6]], "-- Probability:", predicts[0,index[6]])
            cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 225, 0), 2)
            
            
            temp = { "detected": {
                        "class": classes[index[6]],
                        "prob":predicts[0,index[6]],
                        "x": x,
                        "y": y,
                        "w": w,
                        "h": h
                            
                        }}
                          
            output.append(temp)
    

                    
    cv2.imshow('Tracker',frame2)
    
    if cv2.waitKey(30) == 27:
      break
cap.release()
cv2.destroyAllWindows()



def find_preds(predicts):
    actual_pred =[]
    for p in predicts:
        max_i=0
        for i in range(1,len(p)):
            if p[i] > p[max_i]:
                max_i = i
        actual_pred.append(max_i)
    return actual_pred



